In [ ]:
# install deps
!pip install pendulum kafka-python

# download feast message contract
!mkdir -p /home/feast/types

!wget https://raw.githubusercontent.com/gojek/feast/master/protos/feast/types/FeatureRow.proto -P /home/feast/types
!wget https://raw.githubusercontent.com/gojek/feast/master/protos/feast/types/Field.proto -P /home/feast/types
!wget https://raw.githubusercontent.com/gojek/feast/master/protos/feast/types/Value.proto -P /home/feast/types

# compile protos
!protoc -I /home --python_out=/home /home/feast/types/*.proto

In [8]:
import logging
import pendulum
from google.protobuf.timestamp_pb2 import Timestamp
from kafka import KafkaProducer
from feast.types.FeatureRow_pb2 import FeatureRow
from feast.types.Field_pb2 import Field
from feast.types.Value_pb2 import Value, Int32List, BytesList


In [10]:
# build message
row = FeatureRow()
row.fields.MergeFrom([
    Field(name="person_id", value=Value(int64_val=1234)),
    Field(name="gender", value=Value(string_val="M")),
])
ts = Timestamp()
ts.FromJsonString(pendulum.now("UTC").to_iso8601_string())
row.event_timestamp.CopyFrom(ts)
row.feature_set = "proj_8619/person_info:2" # [PROJECT_NAME]/[FEATURE_NAME]:[VERSION]

In [13]:
# send to kafka
producer = KafkaProducer(bootstrap_servers="10.163.12.6:9092")
producer.send("feast-fs-person-predictions", row.SerializeToString())
producer.flush()